In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient()
db = client.linkedin
collection = db.jobIds

In [4]:
df = pd.DataFrame(list(collection.find({'label' : { '$exists': True}} )))
df_test = pd.DataFrame(list(collection.find({'loaded': True,'label' : { '$exists': False}} )))
df_full = pd.DataFrame(list(collection.find({})))

In [5]:
df_test.head(3)

,_id,jobId,loaded,company,city,posted,applicants,applicants_per_day,description,Seniority level,Employment type,Job function,Industries,title
0,5efca850b4432a82d5b96daa,1666204878,True,Michael Page,"Brussels, Brussels Region, Belgium",196,26,0.131980,Financial Planning & Data AnalystHybrid functi...,Associate,Full-time,Information Technology,Information Technology and Services,Financial Planning & Data Analyst
1,5efca850b4432a82d5b96dab,1869872248,True,Amoria Bond,"Brussels, Brussels Region, Belgium",7,80,10.000000,Python DeveloperAmoria Bond is looking for a P...,Mid-Senior level,Contract,Engineering,Information Technology,Senior Python Developer
2,5efca850b4432a82d5b96dad,1907262846,True,Xquisit,"Brussels, Brussels Region, Belgium",21,10,0.454545,Wij zoeken géén Data Scientist of Developer vo...,Entry level,Full-time,Information Technology,Information Technology and Services,Enterprise Data Specialist / Manager


In [6]:
from sklearn.feature_extraction import text
from portuguese_stop_words import pt_br_stopwords
stop_words = text.ENGLISH_STOP_WORDS.union(pt_br_stopwords)

In [7]:
df.head(5)

,_id,jobId,loaded,company,city,posted,applicants,applicants_per_day,description,Seniority level,Employment type,Job function,Industries,title,label
0,5efbc06fad92715989d1c6b7,1905005152,True,SciCrop®,"São Paulo, São Paulo, Brazil",4,10,2.000000,The ideal candidate's favorite words are learn...,Mid-Senior level,Temporary,Engineering,Information Technology,Data Scientist,1
1,5efbc06fad92715989d1c6b8,1687689775,True,Liv Up,"São Paulo, São Paulo, Brazil",168,80,0.473373,A Liv Up é a startup que está construindo a al...,Entry level,Full-time,Engineering,Information Technology,Data Scientist,1
2,5efbc06fad92715989d1c6b9,1866140587,True,Ame Digital,"São Paulo, São Paulo, Brazil",56,178,3.122807,"Oi, somos a Ame!Somos um fintech trabalhando p...",Entry level,Full-time,Engineering,Information Technology,Data Scientist,1
3,5efbc06fad92715989d1c6ba,1866231112,True,Capgemini,"Barueri, São Paulo, Brazil",21,10,0.454545,Quer trabalhar em uma das empresas mais éticas...,Mid-Senior level,Full-time,Analyst,Information Technology,Data Scientist (Teradata),0
4,5efbc06fad92715989d1c6bb,1925461308,True,Dasa,"São Paulo, São Paulo, Brazil",2,150,50.000000,Job DescriptionWith innovation and entrepreneu...,Entry level,Full-time,Engineering,Information Technology,Data Scientist,1


In [8]:
from sklearn.model_selection import train_test_split

df['Description Length'] = df['description'].apply(len)
df_test['Description Length'] = df_test['description'].apply(len)

X = df.drop(columns=['label'])
y = df[['label']]

usedFeatures = ['applicants', 'applicants_per_day', 'Description Length']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

final_features_train = [X_train[usedFeatures]]
final_features_val = [X_val[usedFeatures]]
final_features_test = [df_test[usedFeatures]]

final_vects_train = []
final_vects_val = []
final_vects_test = []

In [15]:

seniority_train = pd.get_dummies(X_train['Seniority level'])
seniority_val = pd.get_dummies(X_val['Seniority level'])
seniority_test = pd.get_dummies(df_test['Seniority level'])

# Get missing columns in the training test
missing_cols = set( seniority_train.columns ) - set( seniority_val.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    seniority_val[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
seniority_val = seniority_val[seniority_train.columns]

# Get missing columns in the training test
missing_cols = set( seniority_train.columns ) - set( seniority_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    seniority_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
seniority_test = seniority_test[seniority_train.columns]

final_features_train.append(seniority_train)
final_features_val.append(seniority_val)
final_features_test.append(seniority_test)

In [17]:
from feature_extractor import get_ohe_features

test = get_ohe_features(X_train, 'Seniority level')

In [18]:
test

,Associate,Director,Entry level,Executive,Internship,Mid-Senior level,Not Applicable
467,0,0,1,0,0,0,0
18,0,0,1,0,0,0,0
264,0,0,0,0,1,0,0
302,1,0,0,0,0,0,0
274,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
334,0,0,1,0,0,0,0
481,0,1,0,0,0,0,0
425,0,0,1,0,0,0,0
279,0,0,1,0,0,0,0


In [34]:
df_full['Industries'].unique()

array(['Information Technology', 'Internet', 'Science',
       'Information Technology and Services', 'Real Estate',
       'Financial Services', 'Marketing and Advertising', 'Engineering',
       'Supply Chain', 'Information Services',
       'Health, Wellness and Fitness', 'Computer Software',
       'Education Management', 'Chemicals', 'Sales', 'Strategy/Planning',
       'Analyst', 'Training', 'Product Management',
       'Management Consulting', 'Staffing and Recruiting', 'Advertising',
       'Paper & Forest Products', 'Market Research',
       'Transportation/Trucking/Railroad', 'Art/Creative',
       'Telecommunications', 'Pharmaceuticals', 'Higher Education',
       'Research', 'Consulting', 'Writing/Editing',
       'Hospital & Health Care', 'Manufacturing', 'General Business',
       'Banking', 'Electrical/Electronic Manufacturing',
       'Food & Beverages', 'Retail', 'Marketing', 'Public Relations',
       'Apparel & Fashion', 'Computer Hardware', 'Design', nan,
       'Me

In [9]:
emp_type_train = pd.get_dummies(X_train['Employment type'])
emp_type_val = pd.get_dummies(X_val['Employment type'])
emp_type_test = pd.get_dummies(df_test['Employment type'])

# Get missing columns in the training test
missing_cols = set( emp_type_train.columns ) - set( emp_type_val.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    emp_type_val[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
emp_type_val = emp_type_val[emp_type_train.columns]

# Get missing columns in the training test
missing_cols = set( emp_type_train.columns ) - set( emp_type_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    emp_type_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
emp_type_test = emp_type_test[emp_type_train.columns]

final_features_train.append(emp_type_train)
final_features_val.append(emp_type_val)
final_features_test.append(emp_type_test)

In [10]:
job_function_columns = ['Administrative', 'Analyst', 'Business Development', 'Consulting',
       'Design', 'Education', 'Engineering', 'Finance', 'Health Care Provider',
       'Human Resources', 'Information Technology', 'Management', 'Marketing',
       'Other', 'Product Management', 'Project Management',
       'Quality Assurance', 'Research', 'Sales', 'Science',
       'Strategy/Planning']

job_function_train = pd.get_dummies(X_train['Job function'])
job_function_val = pd.get_dummies(X_val['Job function'])
job_function_test = pd.get_dummies(df_test['Job function'])

# Get missing columns in the training test
missing_cols = set( job_function_train.columns ) - set( job_function_val.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    job_function_val[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
job_function_val = job_function_val[job_function_train.columns]

# Get missing columns in the training test
missing_cols = set( job_function_train.columns ) - set( job_function_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    job_function_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
job_function_test = job_function_test[job_function_train.columns]

final_features_train.append(job_function_train)
final_features_val.append(job_function_val)
final_features_test.append(job_function_test)

In [11]:
industries_columns = ['Analyst', 'Apparel & Fashion', 'Art/Creative', 'Business Development',
       'Chemicals', 'Computer Hardware', 'Computer Software', 'Consulting',
       'Customer Service', 'Design', 'Education Management',
       'Electrical/Electronic Manufacturing', 'Engineering', 'Finance',
       'Financial Services', 'Food & Beverages',
       'Health, Wellness and Fitness', 'Hospital & Health Care',
       'Information Services', 'Information Technology',
       'Information Technology and Services', 'Internet',
       'Logistics and Supply Chain', 'Manufacturing', 'Market Research',
       'Marketing', 'Marketing and Advertising', 'Medical Devices',
       'Paper & Forest Products', 'Pharmaceuticals', 'Research', 'Retail',
       'Sales', 'Science', 'Staffing and Recruiting', 'Strategy/Planning',
       'Telecommunications', 'Training', 'Writing/Editing']

industries_train = pd.get_dummies(X_train['Industries'])
industries_val = pd.get_dummies(X_val['Industries'])
industries_test = pd.get_dummies(df_test['Industries'])

# Get missing columns in the training test
missing_cols = set( industries_train.columns ) - set( industries_val.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    industries_val[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
industries_val = industries_val[industries_train.columns]

# Get missing columns in the training test
missing_cols = set( industries_train.columns ) - set( industries_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    industries_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
industries_test = industries_test[industries_train.columns]

final_features_train.append(industries_train)
final_features_val.append(industries_val)
final_features_test.append(industries_test)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_vec = TfidfVectorizer(min_df=4, ngram_range=(1,2), stop_words=stop_words)
title_bow_train = title_vec.fit_transform(X_train['title'])
title_bow_val = title_vec.transform(X_val['title'])
title_bow_test = title_vec.transform(df_test['title'])

print(title_bow_train.shape)

final_vects_train.append(title_bow_train)
final_vects_val.append(title_bow_val)
final_vects_test.append(title_bow_test)

(386, 81)


In [13]:
desc_vec = TfidfVectorizer(min_df=4, ngram_range=(1,3), stop_words=stop_words)
desc_bow_train = desc_vec.fit_transform(X_train['description'])
desc_bow_val = desc_vec.transform(X_val['description'])
desc_bow_test = desc_vec.transform(df_test['description'])

print(desc_bow_train.shape)

final_vects_train.append(desc_bow_train)
final_vects_val.append(desc_bow_val)
final_vects_test.append(desc_bow_test)

(386, 7267)


In [14]:
from scipy.sparse import hstack

X_train_full = hstack(final_features_train + final_vects_train)
X_val_full = hstack(final_features_val + final_vects_val)
X_test_full = hstack(final_features_test + final_vects_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier

mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=-1, min_samples_leaf=10)
mdl.fit(X_train_full, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
from sklearn.metrics import roc_auc_score, average_precision_score

p = mdl.predict_proba(X_val_full)[:, 1]

precision = average_precision_score(y_val, p)
auc = roc_auc_score(y_val, p)

print("Precision: {:.2f}%\tAUC: {:.3f}".format(precision*100, auc))

Precision: 51.09%	AUC: 0.799


In [17]:
# Precision: 68.71%	AUC: 0.900 - title_vec = (min_df=3, ngram_range=(1,2) desc_vec =(min_df=4, ngram_range=(1,3)
# Precision: 71.35%	AUC: 0.875 

In [18]:
p = mdl.predict_proba(X_test_full)[:, 1]

In [19]:
df_test['p'] = p

In [20]:
df_test.sort_values('p', ascending=False)[df_test['p'] > 0.8].head(5)

,_id,jobId,loaded,company,city,posted,applicants,applicants_per_day,description,Seniority level,Employment type,Job function,Industries,title,Description Length,p


In [21]:
from lightgbm import LGBMClassifier

In [22]:
mdl = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=-1)
mdl.fit(X_train_full, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [23]:
p = mdl.predict_proba(X_val_full)[:, 1]

precision = average_precision_score(y_val, p)
auc = roc_auc_score(y_val, p)

print("Precision: {:.2f}%\tAUC: {:.3f}".format(precision*100, auc))

Precision: 55.83%	AUC: 0.788


In [24]:
from skopt import forest_minimize

In [25]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    min_df_desc = params[8]
    ngram_range_desc = (1, params[9])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range, stop_words=stop_words)
    title_bow_train = title_vec.fit_transform(X_train['title'])
    title_bow_val = title_vec.transform(X_val['title'])
    
    final_features_train.append(title_bow_train)
    final_features_val.append(title_bow_val)

    desc_vec = TfidfVectorizer(min_df=min_df_desc, ngram_range=ngram_range_desc, stop_words=stop_words)
    desc_bow_train = desc_vec.fit_transform(X_train['description'])
    desc_bow_val = desc_vec.transform(X_val['description'])

    final_features_train.append(desc_bow_train)
    final_features_val.append(desc_bow_val)

    
    
    X_train_full = hstack(final_features_train)
    X_val_full = hstack(final_features_val)

    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(X_train_full, y_train)
    
    p = mdl.predict_proba(X_val_full)[:, 1]
    
    print(roc_auc_score(y_val, p))
    
    return -average_precision_score(y_val, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (2,10), # min_df title
          (1,3), # ngram_range title
          (2,10), # min_df desc
          (1,5)] # ngram_range desc

res = forest_minimize(tune_lgbm, space, random_state=255479, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.0015917412074366864, 1, 1, 0.7162891223936204, 0.07362894136443292, 147, 7, 2, 3, 5]
0.7962607861936721
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.0815
Function value obtained: -0.4756
Current minimum: -0.4756
Iteration No: 2 started. Evaluating function at random point.
[0.004405577997314528, 5, 16, 0.5222531571218747, 0.37454048985593646, 188, 2, 2, 2, 4]
0.783796740172579
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.2665
Function value obtained: -0.4708
Current minimum: -0.4756
Iteration No: 3 started. Evaluating function at random point.
[0.004088470709702419, 4, 11, 0.10800798914439425, 0.6744232993630378, 828, 8, 1, 10, 5]
0.7694151486097794
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.0565
Function value obtained: -0.4446
Current minimum: -0.4756
Iteration No: 4 started. Evaluating function at random point.
[0.07813279834700627, 3, 2, 0.19

KeyboardInterrupt: 

In [91]:
res.x

[0.005677937885284806,
 4,
 1,
 0.9591262059674839,
 0.794014754551486,
 780,
 9,
 1,
 5,
 2]

In [92]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    min_df_desc = params[8]
    ngram_range_desc = (1, params[9])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range, stop_words=stop_words)
    title_bow_train = title_vec.fit_transform(X_train['title'])
    title_bow_val = title_vec.transform(X_val['title'])
    
    final_features_train.append(title_bow_train)
    final_features_val.append(title_bow_val)

    desc_vec = TfidfVectorizer(min_df=min_df_desc, ngram_range=ngram_range_desc, stop_words=stop_words)
    desc_bow_train = desc_vec.fit_transform(X_train['description'])
    desc_bow_val = desc_vec.transform(X_val['description'])

    final_features_train.append(desc_bow_train)
    final_features_val.append(desc_bow_val)

    
    
    X_train_full = hstack(final_features_train)
    X_val_full = hstack(final_features_val)

    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(X_train_full, y_train)
    
    p = mdl.predict_proba(X_val_full)[:, 1]
    
    print(roc_auc_score(y_val, p))
    
    return -average_precision_score(y_val, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (2,10), # min_df title
          (1,3), # ngram_range title
          (2,10), # min_df desc
          (1,5)] # ngram_range desc

res = forest_minimize(tune_lgbm, space, random_state=255479, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.0015917412074366864, 1, 1, 0.7162891223936204, 0.07362894136443292, 147, 7, 2, 3, 5]


TypeError: no supported conversion for types: (dtype('float64'), dtype('uint8'), dtype('int64'), dtype('int64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'))